In [1]:
import torch
import os
import numpy as np
from torch.utils.data import DataLoader
import matplotlib
import matplotlib.pyplot as plt
import import_ipynb
import importance_sampler_poise
import kl_divergence_calculator
import data_preprocessing
from torchvision.utils import save_image
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets
import torchvision.transforms as transforms
from torch.nn import functional as F  #for the activation function
from torchviz import make_dot
from torch.utils.tensorboard import SummaryWriter
from torchvision import datasets, transforms
import torchvision
import umap
import random
import shutil

importing Jupyter notebook from importance_sampler_poise.ipynb
importing Jupyter notebook from kl_divergence_calculator.ipynb
importing Jupyter notebook from data_preprocessing.ipynb


In [29]:
# learning parameters
latent_dim1 = 32
latent_dim2 = 16
batch_size = 10
dim_MNIST   = 784
lr = 1e-4
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
tx = transforms.ToTensor()
PATH = "/home/achint/Practice_code/1_a_new_start/MNIST_SVHN/weights.txt"
MNIST_TRAINING_PATH = "/home/achint/Practice_code/VAE/MNIST/MNIST/processed/training.pt"
SVHN_TRAINING_PATH  = "/home/achint/Practice_code/VAE/SVHN/train_32x32.mat"
MNIST_TEST_PATH     = "/home/achint/Practice_code/VAE/MNIST/MNIST/processed/test.pt"
SVHN_TEST_PATH  = "/home/achint/Practice_code/VAE/SVHN/test_32x32.mat"
SUMMARY_WRITER_PATH = "/home/achint/Practice_code/logs"
#RECONSTRUCTION_PATH = "/home/achint/Practice_code/Synthetic_dataset/POISE_VAE_SVHN_MNIST/reconstructions_experiment/"

CROSS_RECONSTRUCTION_PATH = "/home/achint/Practice_code/1_a_new_start/MNIST_SVHN/reconstructions_experiment/cross_generation/"
JOINT_RECONSTRUCTION_PATH = "/home/achint/Practice_code/1_a_new_start/MNIST_SVHN/reconstructions_experiment/joint_generation/"

In [30]:
## Importing MNIST and SVHN datasets
joint_dataset_train=data_preprocessing.JointDataset(mnist_pt_path=MNIST_TRAINING_PATH,
                             svhn_mat_path=SVHN_TRAINING_PATH)
joint_dataset_test = data_preprocessing.JointDataset(mnist_pt_path=MNIST_TEST_PATH,
                             svhn_mat_path=SVHN_TEST_PATH)

joint_dataset_train_loader = DataLoader(
    joint_dataset_train,
    batch_size=batch_size,
    shuffle=True,
    drop_last=True
)
joint_dataset_test_loader = DataLoader(
    joint_dataset_test,
    batch_size=batch_size,
    shuffle=False,
    drop_last=True
)

In [31]:
class VAE(nn.Module):
    def __init__(self,latent_dim1, latent_dim2, batch_size,use_mse_loss=True):
        super(VAE,self).__init__()
        self.latent_dim1 = latent_dim1
        self.latent_dim2 = latent_dim2
        self.batch_size = batch_size
        self.use_mse_loss = use_mse_loss
        self.n_IW_samples = 20
        self.IS_sampler              = importance_sampler_poise.importance_sampler(self.latent_dim1, self.latent_dim2, self.batch_size)
        ## Encoder set1(MNIST)
        self.set1_enc1 = nn.Linear(in_features = dim_MNIST,out_features = 512)
        self.set1_enc2 = nn.Linear(in_features = 512,out_features = 128)
        self.set1_enc3 = nn.Linear(in_features = 128,out_features = 2*latent_dim1)
        
        ## Decoder set1(MNIST)
        self.set1_dec1 = nn.Linear(in_features = latent_dim1,out_features = 128)
        self.set1_dec2 = nn.Linear(in_features = 128,out_features = 512)
        self.set1_dec3 = nn.Linear(in_features = 512,out_features = dim_MNIST)
        
        ## Encoder set2(SVHN)
        # input size: 3 x 32 x 32
        self.set2_enc1 = nn.Conv2d(in_channels=3, out_channels=2*latent_dim2, kernel_size=4, stride=2, padding=1)
        # size: 32 x 16 x 16
        self.set2_enc2 = nn.Conv2d(in_channels=2*latent_dim2, out_channels=2*latent_dim2, kernel_size=4, stride=2, padding=1)
        # size: 32 x 8 x 8
        self.set2_enc3 = nn.Conv2d(in_channels=2*latent_dim2, out_channels=latent_dim2, kernel_size=4, stride=2, padding=1)
        # size: 16 x 4 x 4    

        ## Decoder set2(SVHN)
        # input size: 16x1x1
        self.set2_dec0 = nn.ConvTranspose2d(in_channels=latent_dim2,out_channels=latent_dim2, kernel_size=4, stride=1, padding=0)
        # input size: 16x4x4
        self.set2_dec1 = nn.ConvTranspose2d(in_channels=latent_dim2,out_channels=2*latent_dim2, kernel_size=3, stride=1, padding=1)
        # size: 32 x 4 x 4
        self.set2_dec2 = nn.ConvTranspose2d(in_channels=2*latent_dim2,out_channels=2*latent_dim2, kernel_size=5, stride=1, padding=0)
        # size: 32 x 8 x 8
        self.set2_dec3 = nn.ConvTranspose2d(in_channels=2*latent_dim2,out_channels=2*latent_dim2, kernel_size=4, stride=2, padding=1)
        # size: 32 x 16 x 16
        self.set2_dec4 = nn.ConvTranspose2d(in_channels=2*latent_dim2,out_channels=3, kernel_size=4, stride=2, padding=1)
        # size: 3 x 32 x 32
        
        self.SVHNc1 = nn.Conv2d(latent_dim2, latent_dim2, 4, 1, 0)
        # size: 16 x 1 x 1
        self.SVHNc2 = nn.Conv2d(latent_dim2, latent_dim2, 4, 1, 0)
        # size: 16 x 1 x 1
        self.register_parameter(name='g11', param = nn.Parameter(torch.randn(latent_dim1,latent_dim2)))
        self.register_parameter(name='g22', param = nn.Parameter(torch.randn(latent_dim1,latent_dim2)))
        self.g12= torch.zeros(latent_dim1,latent_dim2).to(device)
    def weighted_mse_loss(self,weights,reconstruction,data):
        loss = torch.sum(weights * ((data - reconstruction) ** 2).T)
        return loss
    def forward(self,x1,x2,cross_generation,joint_generation):
        data1    = x1 #MNIST
        data2    = x2 #SVHN
        # Modality 1 (MNIST)
        x1       = F.relu(self.set1_enc1(x1))
        x1       = F.relu(self.set1_enc2(x1))  
        x1       = self.set1_enc3(x1).view(-1,2,latent_dim1)  # ->[128,2,32]
        mu1      = x1[:,0,:] # ->[128,32]
        log_var1 = x1[:,1,:] # ->[128,32]
        var1     = -torch.exp(log_var1)           #lambdap_2<0
        # Modality 2 (SVHN)
        x2 = x2.view(-1,3, 32,32) 
        x2 = F.relu(self.set2_enc1(x2))
        x2 = F.relu(self.set2_enc2(x2))
        x2 = F.relu(self.set2_enc3(x2))
        # get 'mu' and 'log_var' for SVHN
        mu2      = (self.SVHNc1(x2).squeeze(3)).squeeze(2)
        log_var2 = (self.SVHNc2(x2).squeeze(3)).squeeze(2)
        var2     = -torch.exp(log_var2)       
        g22      = -torch.exp(self.g22)     
        G1       = torch.cat((self.g11,self.g12),0)
        G2       = torch.cat((self.g12,g22),0)
        G        = torch.cat((G1,G2),1)
        if cross_generation:
            mu2   = torch.zeros_like(mu2)
            var2   = torch.zeros_like(var2)
        elif joint_generation:
            mu1   = torch.zeros_like(mu1)
            var1  = torch.zeros_like(var1)
            mu2   = torch.zeros_like(mu2)
            var2  = torch.zeros_like(var2)
        z1_prior,z2_prior,z1_posterior,z2_posterior,IS_weights_prior,IS_weights_post = self.IS_sampler.calc(G,mu1,var1,mu2,var2,self.n_IW_samples)
        total_reconstruction_loss = 0
        weighted_reconstruction1 = torch.zeros_like(data1)               #[batch_size,2]
        weighted_reconstruction2 = torch.zeros_like(data2)
        for i in range(self.n_IW_samples):
            self.z1_IS_prior     = z1_prior[i]
            self.z2_IS_prior     = z2_prior[i]
            self.z1_IS_posterior = z1_posterior[i]
            self.z2_IS_posterior = (z2_posterior[i].unsqueeze(2)).unsqueeze(3)
            # decoding for MNIST
            x1 = F.relu(self.set1_dec1(self.z1_IS_posterior))
            x1 = self.set1_dec2(x1)
            # decoding for SVHN
            x2 = F.relu(self.set2_dec0(self.z2_IS_posterior))
            x2 = F.relu(self.set2_dec1(x2))
            x2 = F.relu(self.set2_dec2(x2))
            x2 = F.relu(self.set2_dec3(x2))
            self.z2_IS_posterior = self.z2_IS_posterior.squeeze()
#             part_fun0,part_fun1,part_fun2 = self.kl_div.calc(G,self.z1_IS_posterior,self.z2_IS_posterior,self.z1_IS_prior,self.z2_IS_prior,mu1,var1,mu2,var2)
            if self.use_mse_loss:
                reconstruction1 = self.set1_dec3(x1)
                reconstruction2 = (self.set2_dec4(x2)).view(-1,3072)
                MSE1 = self.weighted_mse_loss(IS_weights_post[i,:],reconstruction1, data1)
                MSE2 = self.weighted_mse_loss(IS_weights_post[i,:],reconstruction2, data2)
#             else:
#                 reconstruction1 = torch.sigmoid(self.set1_dec3(x1))
#                 reconstruction2 = torch.sigmoid((self.set2_dec4(x2)).view(-1,3072))
#                 bce_loss = nn.BCELoss(reduction='sum')
#                 MSE1 = bce_loss(reconstruction1, data1)
#                 MSE2 = bce_loss(reconstruction2, data2)
            total_reconstruction_loss = total_reconstruction_loss+MSE1+MSE2
            KLD =torch.zeros_like(total_reconstruction_loss)
            weighted_reconstruction1 = weighted_reconstruction1 + (IS_weights_post[i,:]*reconstruction1.T).T
            weighted_reconstruction2 = weighted_reconstruction2 + (IS_weights_post[i,:]*reconstruction2.T).T

        return self.z1_IS_posterior,self.z2_IS_posterior,weighted_reconstruction1,weighted_reconstruction2,mu1,var1,mu2,var2,total_reconstruction_loss, MSE1, MSE2, KLD

In [32]:
model = VAE(latent_dim1, latent_dim2, batch_size,use_mse_loss=True).to(device)
optimizer = optim.Adam(model.parameters(),lr=lr)
for name, para in model.named_parameters():
    print(name)

g11
g22
set1_enc1.weight
set1_enc1.bias
set1_enc2.weight
set1_enc2.bias
set1_enc3.weight
set1_enc3.bias
set1_dec1.weight
set1_dec1.bias
set1_dec2.weight
set1_dec2.bias
set1_dec3.weight
set1_dec3.bias
set2_enc1.weight
set2_enc1.bias
set2_enc2.weight
set2_enc2.bias
set2_enc3.weight
set2_enc3.bias
set2_dec0.weight
set2_dec0.bias
set2_dec1.weight
set2_dec1.bias
set2_dec2.weight
set2_dec2.bias
set2_dec3.weight
set2_dec3.bias
set2_dec4.weight
set2_dec4.bias
SVHNc1.weight
SVHNc1.bias
SVHNc2.weight
SVHNc2.bias


In [33]:
def test(model,joint_dataloader,epoch,cross_generation,joint_generation):
    latent_repMNIST= []
    latent_repSVHN= []
    label_mnist= []
    label_svhn= []
    state = torch.load(PATH)
    model.load_state_dict(state['state_dict'])
    model.eval()
    running_loss = 0.0
    running_mse1 = 0.0
    running_mse2 = 0.0
    running_kld  = 0.0
    running_loss = 0.0
    with torch.no_grad():
        for i,joint_data in enumerate(joint_dataloader):
            data1   = joint_data[0]
            data1   = data1.float()

            data2  = joint_data[1]
            data2  = data2.float()

            label1 = joint_data[2]
            label2 = joint_data[3]
            
            data1 = data1.to(device)
            data2 = data2.to(device)
            data1 = data1.view(data1.size(0), -1)
            data2 = data2.view(data2.size(0), -1)
            if cross_generation:
                data2   = torch.zeros_like(data2)
            elif joint_generation:
                data1   = torch.zeros_like(data1)
                data2   = torch.zeros_like(data2)
#             data1 = torch.zeros_like(data1)+torch.mean(data1)
#             data2 = torch.zeros_like(data2)+torch.mean(data2)
            z1_posterior,z2_posterior,reconstruction1,reconstruction2,mu1,var1,mu2,var2,loss, MSE1, MSE2, KLD = model(data1,data2,cross_generation,joint_generation)  
            
            running_loss += loss.item()
            running_mse1 += MSE1.item()
            running_mse2 += MSE2.item()
            running_kld  += KLD.item()    
            
            latent_repMNIST.append(z1_posterior)
            latent_repSVHN.append(z2_posterior)
            label_mnist.append(label1)
            label_svhn.append(label2)

            #save the last batch input and output of every epoch
            if i == int(len(joint_dataloader.dataset)/joint_dataloader.batch_size) - 1:
                trans = transforms.Compose([transforms.Resize(32)])
                num_rows = 8
                both = torch.cat((data1.view(batch_size, 1, 28, 28)[:8], 
                                  reconstruction1.view(batch_size, 1, 28, 28)[:8]))
                bothp = torch.cat((data2.view(batch_size, 3, 32, 32)[:8], 
                                  reconstruction2.view(batch_size, 3, 32, 32)[:8]))
                both = torch.cat((both,both,both),axis=1)
                both = trans(both)

                both_single = torch.cat((both,bothp),0)
                if cross_generation: 
                    save_image(both_single.cpu(), os.path.join(CROSS_RECONSTRUCTION_PATH, f"cross_generation_{epoch}.png"), nrow=num_rows)
                elif joint_generation:
                    save_image(both_single.cpu(), os.path.join(JOINT_RECONSTRUCTION_PATH, f"joint_generation_{epoch}.png"), nrow=num_rows)
#                 save_image(both.cpu(), os.path.join(RECONSTRUCTION_PATH, f"1_outputMNIST_{epoch}.png"), nrow=num_rows)
#                 save_image(bothp.cpu(), os.path.join(RECONSTRUCTION_PATH, f"1_outputSVHN_{epoch}.png"), nrow=num_rows)
    test_loss = running_loss/(len(joint_dataloader.dataset))
    mse1_loss = running_mse1 / (len(joint_dataloader.dataset))
    mse2_loss = running_mse2 / (len(joint_dataloader.dataset))
    kld_loss = running_kld / (len(joint_dataloader.dataset))
    latent_repMNIST = torch.vstack(latent_repMNIST).cpu().numpy()
    latent_repSVHN  = torch.vstack(latent_repSVHN).cpu().numpy()
    label_mnist     = torch.hstack(label_mnist).cpu().numpy()
    label_svhn      = torch.hstack(label_svhn).cpu().numpy()
    return test_loss,latent_repMNIST,latent_repSVHN,label_mnist,label_svhn

In [35]:
test_loss = []
epochs = 1
for epoch in range(epochs):
    print(f"Epoch {epoch+1} of {epochs}")
    test_epoch_loss,latent_repMNIST,latent_repSVHN,label_mnist,label_svhn = test(model,
                                                                                 joint_dataset_test_loader,
                                                                                 epoch,
                                                                                cross_generation=False,
                                                                                joint_generation=True)
    test_loss.append(test_epoch_loss)     
    print(f"Test Loss: {test_epoch_loss:.4f}")

Epoch 1 of 1
Test Loss: 698.1569
